###  Name: Shriya Bhat
### Reg: 220968020
### Class: DSE A1
### Week 4 q2

In [4]:
import numpy as np
import random
import matplotlib.pyplot as plt
from time import time

# Heuristic function to calculate makespan (total processing time of the schedule)
def calculate_makespan(schedule, job_processing_times):
    machine_times = np.zeros(len(schedule))
    for i, machine_jobs in enumerate(schedule):
        machine_times[i] = sum(job_processing_times[job] for job in machine_jobs)
    return max(machine_times)

# Randomly generate an initial schedule (random assignment of jobs to machines)
def generate_random_schedule(num_jobs, num_machines):
    schedule = [[] for _ in range(num_machines)]
    for job in range(num_jobs):
        machine = random.randint(0, num_machines - 1)
        schedule[machine].append(job)
    return schedule

# Simple Hill Climbing Algorithm
def simple_hill_climbing(schedule, job_processing_times):
    current_schedule = schedule
    current_makespan = calculate_makespan(current_schedule, job_processing_times)
    
    while True:
        # Generate a neighbor schedule by swapping jobs between machines
        neighbor_schedule = current_schedule.copy()
        machine1, machine2 = random.sample(range(len(schedule)), 2)
        if len(neighbor_schedule[machine1]) > 0 and len(neighbor_schedule[machine2]) > 0:
            job1 = random.choice(neighbor_schedule[machine1])
            job2 = random.choice(neighbor_schedule[machine2])
            neighbor_schedule[machine1].remove(job1)
            neighbor_schedule[machine2].remove(job2)
            neighbor_schedule[machine1].append(job2)
            neighbor_schedule[machine2].append(job1)

        # Evaluate the neighbor solution
        neighbor_makespan = calculate_makespan(neighbor_schedule, job_processing_times)
        
        # If the neighbor is better, move to it
        if neighbor_makespan < current_makespan:
            current_schedule = neighbor_schedule
            current_makespan = neighbor_makespan
        else:
            break
    return current_schedule, current_makespan

# Stochastic Hill Climbing
def stochastic_hill_climbing(schedule, job_processing_times):
    current_schedule = schedule
    current_makespan = calculate_makespan(current_schedule, job_processing_times)
    
    while True:
        # Generate a neighbor schedule (random job swap)
        neighbor_schedule = current_schedule.copy()
        machine1, machine2 = random.sample(range(len(schedule)), 2)
        job1 = random.choice(neighbor_schedule[machine1])
        job2 = random.choice(neighbor_schedule[machine2])
        neighbor_schedule[machine1].remove(job1)
        neighbor_schedule[machine2].remove(job2)
        neighbor_schedule[machine1].append(job2)
        neighbor_schedule[machine2].append(job1)

        # Evaluate the neighbor solution
        neighbor_makespan = calculate_makespan(neighbor_schedule, job_processing_times)
        
        # Move to the neighbor based on probability
        if neighbor_makespan < current_makespan or random.random() < np.exp((current_makespan - neighbor_makespan) / 100):
            current_schedule = neighbor_schedule
            current_makespan = neighbor_makespan
        else:
            break
    return current_schedule, current_makespan

# Steepest Ascent Hill Climbing
# Steepest Ascent Hill Climbing with proper job swapping
def steepest_ascent_hill_climbing(schedule, job_processing_times):
    current_schedule = schedule
    current_makespan = calculate_makespan(current_schedule, job_processing_times)

    while True:
        best_neighbor = None
        best_makespan = current_makespan

        # Generate and evaluate all neighbor schedules
        for machine1 in range(len(schedule)):
            for machine2 in range(len(schedule)):
                if machine1 != machine2:
                    for job1 in schedule[machine1]:
                        for job2 in schedule[machine2]:
                            # Ensure both jobs are present on the respective machines before removing
                            if job1 in schedule[machine1] and job2 in schedule[machine2]:
                                # Create a copy of the current schedule to avoid modifying it during iteration
                                neighbor_schedule = [machine_jobs.copy() for machine_jobs in current_schedule]
                                
                                # Swap jobs between machines
                                if job1 in neighbor_schedule[machine1]:
                                    neighbor_schedule[machine1].remove(job1)
                                if job2 in neighbor_schedule[machine2]:
                                    neighbor_schedule[machine2].remove(job2)
                                
                                neighbor_schedule[machine1].append(job2)
                                neighbor_schedule[machine2].append(job1)

                                # Calculate makespan of the new neighbor schedule
                                neighbor_makespan = calculate_makespan(neighbor_schedule, job_processing_times)

                                # If the new neighbor is better (lower makespan), update
                                if neighbor_makespan < best_makespan:
                                    best_neighbor = neighbor_schedule
                                    best_makespan = neighbor_makespan

        # If a better neighbor was found, update current schedule and makespan
        if best_neighbor:
            current_schedule = best_neighbor
            current_makespan = best_makespan
        else:
            break  # No improvement found, exit the loop

    return current_schedule, current_makespan


# Random Restart Hill Climbing
def random_restart_hill_climbing(num_restarts, num_jobs, num_machines, job_processing_times):
    best_schedule = None
    best_makespan = float('inf')
    
    for _ in range(num_restarts):
        initial_schedule = generate_random_schedule(num_jobs, num_machines)
        schedule, makespan = simple_hill_climbing(initial_schedule, job_processing_times)
        
        if makespan < best_makespan:
            best_schedule = schedule
            best_makespan = makespan
    
    return best_schedule, best_makespan

# Main function to evaluate the algorithms
def evaluate_algorithms(num_jobs, num_machines, job_processing_times):
    # Example: Run the algorithms and compare
    start_time = time()
    schedule, makespan = simple_hill_climbing(generate_random_schedule(num_jobs, num_machines), job_processing_times)
    print(f"Simple Hill Climbing: Makespan = {makespan}, Time = {time() - start_time}s")
    
    start_time = time()
    schedule, makespan = stochastic_hill_climbing(generate_random_schedule(num_jobs, num_machines), job_processing_times)
    print(f"Stochastic Hill Climbing: Makespan = {makespan}, Time = {time() - start_time}s")
    
    start_time = time()
    schedule, makespan = steepest_ascent_hill_climbing(generate_random_schedule(num_jobs, num_machines), job_processing_times)
    print(f"Steepest Ascent Hill Climbing: Makespan = {makespan}, Time = {time() - start_time}s")
    
    start_time = time()
    schedule, makespan = random_restart_hill_climbing(10, num_jobs, num_machines, job_processing_times)
    print(f"Random Restart Hill Climbing: Makespan = {makespan}, Time = {time() - start_time}s")

# Example usage
num_jobs = 50
num_machines = 10
job_processing_times = np.random.randint(1, 100, num_jobs)

evaluate_algorithms(num_jobs, num_machines, job_processing_times)


Simple Hill Climbing: Makespan = 409.0, Time = 0.0s
Stochastic Hill Climbing: Makespan = 544.0, Time = 0.0s
Steepest Ascent Hill Climbing: Makespan = 327.0, Time = 0.16119909286499023s
Random Restart Hill Climbing: Makespan = 340.0, Time = 0.0s


In [9]:
import random
import time

# Function to generate a random initial schedule
def generate_initial_schedule(num_jobs, num_machines):
    schedule = [[] for _ in range(num_machines)]
    for job in range(num_jobs):
        machine = random.randint(0, num_machines - 1)
        schedule[machine].append(job)
    return schedule

# Function to calculate the total processing time of a schedule
def calculate_processing_time(schedule, job_times):
    machine_times = [sum(job_times[job] for job in machine) for machine in schedule]
    return max(machine_times)

# Simple Hill Climbing
def simple_hill_climbing(num_jobs, num_machines, job_times, max_iterations=1000):
    schedule = generate_initial_schedule(num_jobs, num_machines)
    best_schedule = schedule
    best_time = calculate_processing_time(schedule, job_times)
    
    for _ in range(max_iterations):
        # Generate a neighboring solution by moving a random job to another machine
        new_schedule = [list(machine) for machine in schedule]
        job = random.randint(0, num_jobs - 1)
        source_machine = random.randint(0, num_machines - 1)
        target_machine = random.randint(0, num_machines - 1)
        
        if job in new_schedule[source_machine]:
            new_schedule[source_machine].remove(job)
            new_schedule[target_machine].append(job)
        
        new_time = calculate_processing_time(new_schedule, job_times)
        if new_time < best_time:
            best_time = new_time
            best_schedule = new_schedule
    
    return best_schedule, best_time

# Stochastic Hill Climbing
def stochastic_hill_climbing(num_jobs, num_machines, job_times, max_iterations=1000):
    schedule = generate_initial_schedule(num_jobs, num_machines)
    best_schedule = schedule
    best_time = calculate_processing_time(schedule, job_times)
    
    for _ in range(max_iterations):
        # Generate a neighboring solution randomly
        new_schedule = [list(machine) for machine in schedule]
        job = random.randint(0, num_jobs - 1)
        source_machine = random.randint(0, num_machines - 1)
        target_machine = random.randint(0, num_machines - 1)
        
        if job in new_schedule[source_machine]:
            new_schedule[source_machine].remove(job)
            new_schedule[target_machine].append(job)
        
        new_time = calculate_processing_time(new_schedule, job_times)
        if new_time < best_time:
            best_time = new_time
            best_schedule = new_schedule
    
    return best_schedule, best_time

# Steepest Ascent Hill Climbing
def steepest_ascent_hill_climbing(num_jobs, num_machines, job_times, max_iterations=1000):
    schedule = generate_initial_schedule(num_jobs, num_machines)
    best_schedule = schedule
    best_time = calculate_processing_time(schedule, job_times)
    
    for _ in range(max_iterations):
        best_neighbor_time = best_time
        best_neighbor = best_schedule
        
        # Evaluate all neighbors
        for source_machine in range(num_machines):
            for target_machine in range(num_machines):
                for job in schedule[source_machine]:
                    new_schedule = [list(machine) for machine in schedule]
                    new_schedule[source_machine].remove(job)
                    new_schedule[target_machine].append(job)
                    
                    new_time = calculate_processing_time(new_schedule, job_times)
                    if new_time < best_neighbor_time:
                        best_neighbor_time = new_time
                        best_neighbor = new_schedule
        
        # If no improvement, break early
        if best_neighbor_time >= best_time:
            break
        best_time = best_neighbor_time
        best_schedule = best_neighbor
    
    return best_schedule, best_time

# Compare algorithms
def compare_algorithms(num_jobs, num_machines, job_times):
    algorithms = [
        ("Simple Hill Climbing", simple_hill_climbing),
        ("Stochastic Hill Climbing", stochastic_hill_climbing),
        ("Steepest Ascent Hill Climbing", steepest_ascent_hill_climbing)
    ]
    
    for name, algorithm in algorithms:
        start_time = time.time()
        best_schedule, best_time = algorithm(num_jobs, num_machines, job_times)
        execution_time = time.time() - start_time
        print(f"{name}: Best Time = {best_time}, Execution Time = {execution_time:.4f} seconds")

# Sample job times (random values between 1 and 10 for demonstration purposes)
num_jobs = 50
num_machines = 10
job_times = [random.randint(1, 10) for _ in range(num_jobs)]

compare_algorithms(num_jobs, num_machines, job_times)


Simple Hill Climbing: Best Time = 46, Execution Time = 0.0110 seconds
Stochastic Hill Climbing: Best Time = 40, Execution Time = 0.0069 seconds
Steepest Ascent Hill Climbing: Best Time = 44, Execution Time = 0.0050 seconds
